In [3]:
import re
from datetime import datetime
from astropy.time import Time
import csv
import MySQLdb as mariadb

Best practice way to fully update the database each time

1. Create a new table
2. Load CSV file into new table
3. Drop existing old table if exists
4. rename current table to old (or could just remove)
5. Rename new table to original

In [51]:
db_name = "flexbuff_test"
conn = mariadb.connect(user='auscope', passwd='password')
cursor = conn.cursor()
query = "CREATE DATABASE IF NOT EXISTS " + db_name +";"
cursor.execute(query)
conn.commit()
query = "USE " + db_name
cursor.execute(query)
conn.commit()

In [52]:
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
for ant in station_id:
    query = "CREATE TABLE IF NOT EXISTS "+ ant + "_NEW (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, DataUsage BIGINT);" 
    cursor.execute(query)
    conn.commit()
conn.close()

In [53]:
conn = mariadb.connect(user='auscope', passwd='password', db=str(db_name))
query = """LOAD DATA LOCAL INFILE '/home/tiege/Documents/research/auscope/flexbuff_database/disk_used_test.txt'
        REPLACE
        INTO TABLE flexbuff_test.Ho_NEW
        FIELDS TERMINATED BY ','
        LINES TERMINATED BY '\n'
        (ExpID, DataUsage);"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

# this adds the CSV file data to a given table - it adds things that are new - but does not remove things that no longer exist.


In [54]:
conn = mariadb.connect(user='auscope', passwd='password', db=str(db_name))
cursor = conn.cursor()
for ant in station_id:
    query = "DROP TABLE IF EXISTS " + ant + "_OLD;"
    cursor.execute(query)
    conn.commit()

In [55]:
conn = mariadb.connect(user='auscope', passwd='password', db=str(db_name))
cursor = conn.cursor()
for ant in station_id:
    query = "ALTER TABLE " + ant + " RENAME TO " + ant + "_OLD;"
    cursor.execute(query)
    conn.commit()


In [56]:
conn = mariadb.connect(user='auscope', passwd='password', db=str(db_name))
cursor = conn.cursor()
for ant in station_id:
    query = "ALTER TABLE " + ant + "_NEW RENAME TO " + ant + ";"
    cursor.execute(query)
    conn.commit()


In [57]:
#db_name = "flexbuff_test"
#conn = mariadb.connect(user='auscope', passwd='password', db=str(db_name))
#with open('disk_used.txt', newline='') as csvfile:
#    disk_usage = csv.reader(csvfile, delimiter=',') 
#    for row in disk_usage:
#        sql_command = "INSERT IGNORE INTO {} (ExpID, DataUsage) VALUES (%s, %s);".format("Ke")
#        data = [row[0], int(row[1])]      
#        cursor = conn.cursor()
#        cursor.execute(sql_command, data)
#        conn.commit()
#    conn.close()

Try make loop more efficient:

In [70]:
# Create database if it doesn't exist.
db_name = "flexbuff_test"
conn = mariadb.connect(user='auscope', passwd='password')
cursor = conn.cursor()
query = "CREATE DATABASE IF NOT EXISTS " + db_name +";"
cursor.execute(query)
conn.commit()
query = "USE " + db_name
cursor.execute(query)
conn.commit()

flexbuff_id = ['flexbuffke', 'flexbuffhb']
for flexbuff in flexbuff_id:
    # This process seems somewhat redundant but is best practice for ensuring table data is not deleted before ensuring new data has been loaded without errors.
    # Create a table for the new inbound data named flexbuffXX_NEW
    query = "CREATE TABLE IF NOT EXISTS "+ flexbuff + "_NEW (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, DataUsage BIGINT);" 
    cursor.execute(query)
    conn.commit()
    # Create a temporary 'current' flexbuffXX table, this table will only not exist on a first run
    query = "CREATE TABLE IF NOT EXISTS "+ flexbuff + " (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, DataUsage BIGINT);" 
    cursor.execute(query)
    conn.commit()
    # Load CSV data into the newly created flexbuffXX_NEW table
    query = "LOAD DATA LOCAL INFILE '/home/tiege/Documents/research/auscope/flexbuff_database/disk_used_" + flexbuff + ".csv' REPLACE INTO TABLE "+ flexbuff + "_NEW FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' (ExpID, DataUsage);"
    cursor.execute(query)
    conn.commit()
    # Drop any existing flexbuffXX_OLD table (this will soon be replaced by the current flexbuffXX table)
    query = "DROP TABLE IF EXISTS " + flexbuff + "_OLD;"
    cursor.execute(query)
    conn.commit()
    # Rename current flexbuffXX table that is getting replaced to flexbuffXX_OLD
    query = "ALTER TABLE " + flexbuff + " RENAME TO " + flexbuff + "_OLD;"
    cursor.execute(query)
    conn.commit()
    # Rename the new data table flexbuffXX_NEW to flexbuffXX
    query = "ALTER TABLE " + flexbuff + "_NEW RENAME TO " + flexbuff + ";"
    cursor.execute(query)
    conn.commit()
    

Now I need to build in an scp component to grab the disk_usage files from flexbuff machines and rename them.

In [4]:
from paramiko import SSHClient
from scp import SCPClient

In [3]:
sshHb = SSHClient()
sshHb.load_system_host_keys()
sshHb.connect(hostname='flexbuffhb.phys.utas.edu.au', 
            #port = 'port',
            username='observer',
            #password=''
            #pkey='load_key_if_relevant'
           )

scpHb = SCPClient(sshHb.get_transport())
scpHb.get('/tmp/disk_used.txt', 'disk_used_flexbuffhb.csv')
scpHb.close()
sshHb.close()

sshKe = SSHClient()
sshKe.load_system_host_keys()
sshKe.connect(hostname='flexbuffke.phys.utas.edu.au', 
            #port = 'port',
            username='observer',
            #password=''
            #pkey='load_key_if_relevant'
           )

scpKe = SCPClient(sshKe.get_transport())
scpKe.get('/tmp/disk_used.txt', 'disk_used_flexbuffke.csv')
scpKe.close()
sshKe.close()

This works well but requires the password in plain text in the code. Need to figure out a way around this.

^ sshkeys work

Time to start writing the functions for the final script

In [5]:
def flexbuffContentsPull(flexbuff_tag): # function for scp of disk_used.txt file from a given flexxbuff machine to cwd.
    flexbuff_tag = str(flexbuff_tag)
    ssh = SSHClient()
    ssh.load_system_host_keys()
    ssh.connect(hostname= flexbuff_tag + '.phys.utas.edu.au', 
                  username='observer',
                  #password=''
                 )

    scp = SCPClient(ssh.get_transport())
    scp.get('/tmp/disk_used.txt', 'disk_used_' + flexbuff_tag + '.csv')
    scp.close()
    ssh.close()

In [12]:
def connectFlexbuffDB(db_name): # connects to an existing flexbuff SQL database or creates a new one if it doesn't exist.
    db_name = str(db_name)
    conn = mariadb.connect(user='auscope', passwd='password')
    cursor = conn.cursor()
    query = "CREATE DATABASE IF NOT EXISTS " + db_name +";"
    cursor.execute(query)
    conn.commit()
    query = "USE " + db_name
    cursor.execute(query)
    conn.commit()
# this function isnt really useful as I will need to reconnect again to add shit. May aswell write the initial connection into the main() function

In [16]:
def updateFlexbuffContents(flexbuff_tag, db_name): # add data from disk_used file into the SQL database.
    flexbuff_tag = str(flexbuff_tag)
    db_name = str(db_name)
    # connect to the database
    conn = mariadb.connect(user='auscope', passwd='password', db=db_name)
    cursor = conn.cursor()
    # This process seems somewhat redundant but is best practice for ensuring table data is not deleted before ensuring new data has been loaded without errors.
    # Create a table for the new inbound data named flexbuffXX_NEW
    query = "CREATE TABLE IF NOT EXISTS "+ flexbuff_tag + "_NEW (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, DataUsage BIGINT);" 
    cursor.execute(query)
    conn.commit()
    # Create a temporary 'current' flexbuffXX table, this table will only not exist on a first run
    query = "CREATE TABLE IF NOT EXISTS "+ flexbuff_tag + " (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, DataUsage BIGINT);" 
    cursor.execute(query)
    conn.commit()
    # Load CSV data into the newly created flexbuffXX_NEW table
    query = "LOAD DATA LOCAL INFILE '/home/tiege/Documents/research/auscope/flexbuff_database/disk_used_" + flexbuff_tag + ".csv' REPLACE INTO TABLE "+ flexbuff_tag + "_NEW FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' (ExpID, DataUsage);"
    cursor.execute(query)
    conn.commit()
    # Drop any existing flexbuffXX_OLD table (this will soon be replaced by the current flexbuffXX table)
    query = "DROP TABLE IF EXISTS " + flexbuff_tag + "_OLD;"
    cursor.execute(query)
    conn.commit()
    # Rename current flexbuffXX table that is getting replaced to flexbuffXX_OLD
    query = "ALTER TABLE " + flexbuff_tag + " RENAME TO " + flexbuff_tag + "_OLD;"
    cursor.execute(query)
    conn.commit()
    # Rename the new data table flexbuffXX_NEW to flexbuffXX
    query = "ALTER TABLE " + flexbuff_tag + "_NEW RENAME TO " + flexbuff_tag + ";"
    cursor.execute(query)
    conn.commit()


In [17]:
def main(db_name):
    db_name = str(db_name)
    # create/connect to mariaDB flexbuff database
    print('Connecting to mariaDB database ' + db_name + '.')
    conn = mariadb.connect(user='auscope', passwd='password')
    cursor = conn.cursor()
    query = "CREATE DATABASE IF NOT EXISTS " + db_name +";"
    cursor.execute(query)
    conn.commit()
    # setup the tags for the relevant flexbuff machines
    flexbuff_id = ['flexbuffhb', 'flexbuffke', 'flexbuffyg', 'flexbuffcd']
    # for each flexbuff machine, pull relevant disk usage data and add to the database
    for flexbuff in flexbuff_id:
        print('Pulling disk contents file from ' + flexbuff + '.')
        flexbuffContentsPull(flexbuff)
        print('Adding disk contents to the ' + flexbuff + ' table of the ' + db_name + ' mariaDB database.')
        updateFlexbuffContents(flexbuff, db_name)

In [18]:
main('flexbuff_test')

Connecting to mariaDB database flexbuff_test.
Pulling disk contents file from flexbuffhb.
Adding disk contents to the flexbuffhb table of the flexbuff_test mariaDB database.
Pulling disk contents file from flexbuffke.
Adding disk contents to the flexbuffke table of the flexbuff_test mariaDB database.
Pulling disk contents file from flexbuffyg.
Adding disk contents to the flexbuffyg table of the flexbuff_test mariaDB database.
Pulling disk contents file from flexbuffcd.
Adding disk contents to the flexbuffcd table of the flexbuff_test mariaDB database.
